# Doc2Vecの動作確認（bot_id=11）


実行条件は下記の通りとします。

- すべての教師データのタグをユニークにするため、[ラベル]\_[連番] 形式のタグを付与


- 一部品詞を落す（Nlangクラスの仕様に従う）


- DBOWを使用


- feature数は、実質的な教師データのパターン数と整合させる（ユーザーさんによっては、回答ラベル数が必ずしも回答パターン数と合致しないため、ラベル数の２倍程度の値を設定するのを目安とします）


- 学習時の反復回数を [feature数 * 10] に設定


現在テストデータとして用意されているBot（ID=11：benefitone.csv）で動作確認を行います。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

## (2) 学習／予測処理

### (2-1) コーパス生成

コーパス（単語が半角スペースで区切られた文字列）生成時、一部の品詞を落とすようにします。

（＝learning.core.nlang.Nlang クラスの仕様に従います）

In [2]:
import numpy as np

from learning.core.learn.learning_parameter import LearningParameter
from learning.core.datasource import Datasource

_bot_id = 11
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': 0.5,
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    'params_for_algorithm': {'C': 140},
    'excluded_labels_for_fitting': None
}

learning_parameter = LearningParameter(attr)

In [3]:
_datasource = Datasource(type='csv')
learning_training_messages = _datasource.learning_training_messages(_bot_id)
questions = np.array(learning_training_messages['question'])
answer_ids = np.array(learning_training_messages['answer_id'])

2017/05/22 AM 12:04:59 ['./fixtures/learning_training_messages/benefitone.csv', './fixtures/learning_training_messages/ptna.csv', './fixtures/learning_training_messages/septeni.csv', './fixtures/learning_training_messages/toyotsu_human.csv']
2017/05/22 AM 12:04:59 ['./fixtures/question_answers/toyotsu_human.csv']


In [4]:
from learning.core.nlang import Nlang

_sentences = np.array(questions)
_separated_sentences = Nlang.batch_split(_sentences)

### (2-2) タグ付け

ラベル毎の教師データ数を制限した学習セットを構築します。

In [5]:
from gensim import models
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

class Doc2VecTrainingSet:
    def __init__(self):
        self.answer_id_count = {}
        self.selected_answer_ids = []
        self.selected_questions = []
        self.selected_separated_sentences = []

    def build(self, questions, answer_ids, separated_sentences):
        for index, answer_id in enumerate(answer_ids):
            if answer_id not in self.answer_id_count.keys():
                self.answer_id_count[answer_id] = 0

            '''
                データをリストに格納
                ラベルはユニークにする必要があるので、
                [answer_id]_[連番] の形式で編集
            '''
            self.selected_answer_ids.append('%04d_%02d' % (
                answer_id,
                self.answer_id_count[answer_id]
            ))
            self.selected_questions.append(questions[index])
            self.selected_separated_sentences.append(separated_sentences[index])
            self.answer_id_count[answer_id] += 1
        
        return self
    
    def get_tagged_document_list(self):
        '''
            ユニークになったコーパス／ラベルから、
            学習セットを生成する
        '''
        tagged_document_list = self.__corpus_to_sentences(
            self.selected_separated_sentences, 
            self.selected_answer_ids
        )

        return tagged_document_list

    def __get_tagged_document(self, sentences, name):
        '''
            models.doc2vecの仕様に従い
            コーパスにタグ付け
        '''
        words = sentences.split(' ')
        return TaggedDocument(words=words, tags=[name])

    def __corpus_to_sentences(self, separated_sentences, answer_ids):
        '''
            TaggedDocumentを生成し、リストに格納
        '''
        tagged_document_list = []
        for idx, (doc, name) in enumerate(zip(separated_sentences, answer_ids)):
            tagged_document = self.__get_tagged_document(doc, name)
            tagged_document_list.append(tagged_document)
            
        return tagged_document_list

In [6]:
'''
    ユニークになったコーパス／ラベルから、
    学習セットを生成する
    
    ラベル数＝88、サンプル数＝7,083
'''
d2v_training_set = Doc2VecTrainingSet()
d2v_training_set.build(questions, answer_ids, _separated_sentences)

### (2-3) 学習処理／モデルのシリアライズ

In [7]:
def doc2vec_model_path(bot_id):
    model_path = 'prototype/better_algorithm/doc2vec.bot%02d.model' % bot_id

    return model_path

def train_by_doc2vec(bot_id, doc2vec_training_set):
    '''
        パラメータ：
         学習時にDBOWを使用する
         featureの数を回答パターン数の２倍（仮決め）に設定
         反復回数をfeature数の10倍に設定
    '''
    sentence_list = doc2vec_training_set.get_tagged_document_list()
    n_pattern = len(doc2vec_training_set.answer_id_count)
    print('train_by_doc2vec: Train data sample=%d, Train data pattern=%d' % (len(sentence_list), n_pattern))

    n_feature = n_pattern * 2
    n_iter = n_feature * 10
    print('train_by_doc2vec: Feature size=%d, Max iteration count=%d' % (n_feature, n_iter))

    # ボキャブラリ生成／学習実行
    model = Doc2Vec(dm=0, size=n_feature, min_count=1, iter=n_iter)
    model.build_vocab(sentence_list)
    ret = model.train(sentence_list)

    '''
        モデル内に保持されているベクトルの数を取得
        （featureの数 [回答パターン数の２倍] と同じであることを確認）
    '''
    if len(model.docvecs) != len(sentence_list):
        raise Exception('train_by_doc2vec: Failed to create document vector')

    # 学習モデルは、ファイルに保存しておく
    model.save(doc2vec_model_path(bot_id))
    print('train_by_doc2vec: document vector size=%d, return=%d' % (len(model.docvecs), ret))

    return ret

In [8]:
'''
    生成された学習セット（タグ付きドキュメント）を
    使用し、学習実行
'''
train_by_doc2vec(_bot_id, d2v_training_set)

train_by_doc2vec: Train data sample=7083, Train data pattern=88
train_by_doc2vec: Feature size=176, Max iteration count=1760
train_by_doc2vec: document vector size=7083, return=45088851


45088851

## (3) accuracy 測定

In [9]:
def predict_similarity(separated_sentence):
    corpus = separated_sentence.split()
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    answer_id, similarity = ret[0]
    return corpus, answer_id, similarity

def get_prediction_statistics(separated_sentences, answer_ids):
    '''
        学習セットの質問文をそのまま予測処理にかけて、
        回答を予測
    '''
    statistics = []
    for i, _ in enumerate(separated_sentences):
        sentence = separated_sentences[i]
        preferred_answer_id = answer_ids[i]
        corpus, answer_id, similarity = predict_similarity(separated_sentences[i])
        corpus_len = len(corpus)
        statistics.append((i, corpus_len, preferred_answer_id, answer_id, similarity))

    return statistics

In [10]:
def calculate_accuracy(prediction_statistics):
    '''
        予測結果を、質問文の単語数毎／回答ID毎に統計する
    '''
    ncorrect_by_corpus_len = {}
    nsample_by_corpus_len = {}

    ncorrect = 0
    nsample = 0

    for statistics in prediction_statistics:
        i, corpus_len, preferred_answer_id, answer_id, similarity = statistics

        '''
            質問文の単語数ごとに統計を取る
        '''
        if corpus_len not in nsample_by_corpus_len.keys():
            ncorrect_by_corpus_len[corpus_len] = 0
            nsample_by_corpus_len[corpus_len] = 0
        nsample_by_corpus_len[corpus_len] += 1

        '''
            正解かどうか検査
            （NNNN_nn 形式ラベルの上４桁が一致していれば正解とします）
        '''
        nsample += 1
        if preferred_answer_id[0:4] == answer_id[0:4]:
            ncorrect += 1
            ncorrect_by_corpus_len[corpus_len] += 1
    
    '''
        全体の正解率
    '''
    accuracy = ncorrect / nsample
    print("accuracy=%0.3f (%d/%d)" % (accuracy, ncorrect, nsample))

    '''
        質問文の単語数ごとの統計情報を編集
    '''
    info_by_corpus_len = []
    for k, v in ncorrect_by_corpus_len.items():
        info_by_corpus_len.append((
            k, 
            ncorrect_by_corpus_len[k]/nsample_by_corpus_len[k], 
            ncorrect_by_corpus_len[k], 
            nsample_by_corpus_len[k]
        ))

    '''
        質問文の単語数ごとの正解率をリスト
    '''
    print("Accuracy info by word count...")
    for info in info_by_corpus_len:
        print("word_count=%2d: accuracy=%0.3f (%d/%d)" % (
            info[0], info[1], info[2], info[3]
        ))

    return accuracy

In [11]:
loaded_model = models.Doc2Vec.load(doc2vec_model_path(_bot_id))

selected_separated_sentences = d2v_training_set.selected_separated_sentences
selected_answer_ids = d2v_training_set.selected_answer_ids

In [12]:
prediction_statistics = get_prediction_statistics(selected_separated_sentences, selected_answer_ids)
calculate_accuracy(prediction_statistics)

accuracy=0.946 (6703/7083)
Accuracy info by word count...
word_count= 1: accuracy=0.953 (162/170)
word_count= 2: accuracy=0.944 (1417/1501)
word_count= 3: accuracy=0.935 (2257/2415)
word_count= 4: accuracy=0.943 (1590/1687)
word_count= 5: accuracy=0.973 (728/748)
word_count= 6: accuracy=0.971 (336/346)
word_count= 7: accuracy=0.979 (137/140)
word_count= 8: accuracy=1.000 (60/60)
word_count= 9: accuracy=1.000 (8/8)
word_count=10: accuracy=1.000 (6/6)
word_count=11: accuracy=1.000 (2/2)


0.9463504164901878

## (4) 予測処理

In [13]:
def predict(word, bot_id):
    '''
        予測処理にかけるコーパスを生成
        （学習セット作成時と同じ関数を使用）
    '''
    corpus = Nlang.split(word).split()

    '''
        コーパスからベクトルを生成し、
        ロードしたモデルから類似ベクトルを検索
    '''
    loaded_model = models.Doc2Vec.load(doc2vec_model_path(bot_id))
    inferred_vector = loaded_model.infer_vector(corpus)
    ret = loaded_model.docvecs.most_similar([inferred_vector])

    return corpus, ret

### (4-1) nosetestsにあった質問文

正しく回答することができているようです。

In [14]:
'''
    契約書を見たいのですが（正解＝4683）
'''
predict('契約書を見たいのですが', _bot_id)

(['契約', '書', '見る'],
 [('4683_12', 0.8393585681915283),
  ('4683_25', 0.8343852758407593),
  ('4683_24', 0.8337586522102356),
  ('4683_00', 0.8298937082290649),
  ('4683_09', 0.7951401472091675),
  ('4683_21', 0.7864588499069214),
  ('4683_13', 0.702258288860321),
  ('4683_15', 0.7021685242652893),
  ('4683_23', 0.7002915143966675),
  ('4683_01', 0.6947280168533325)])

In [15]:
'''
    EXカードを貸してください（正解＝4678）
'''
predict('EXカードを貸してください', _bot_id)

(['ＥＸ', 'カード', '貸す'],
 [('4678_692', 0.797877311706543),
  ('4678_2700', 0.7967407703399658),
  ('4678_2694', 0.7926852703094482),
  ('4678_686', 0.7879177927970886),
  ('4678_2213', 0.685415506362915),
  ('4678_4221', 0.6775147318840027),
  ('4678_2695', 0.6713133454322815),
  ('4678_687', 0.6649485230445862),
  ('4678_2698', 0.652132511138916),
  ('4678_694', 0.649863600730896)])

### (4-2) 教師データに存在する、単語１語だけの質問文

三脚が含まれる質問文が候補としてリストされます。類似度が0.5を上回る結果となります。

（ちなみに確認したところ、4740と4741は同じ「三脚を借りたいのですね。総務部に・・・」という回答文でした）

In [16]:
'''
    三脚（正解＝4740）
'''
predict('三脚', _bot_id)

(['三脚'],
 [('4740_01', 0.7236194610595703),
  ('4740_13', 0.7219536900520325),
  ('4740_00', 0.7217124104499817),
  ('4740_16', 0.7133685350418091),
  ('4740_19', 0.712550163269043),
  ('4740_07', 0.7086933851242065),
  ('4740_08', 0.7081842422485352),
  ('4740_20', 0.7081177234649658),
  ('4740_25', 0.7065144777297974),
  ('4740_04', 0.7038346529006958)])

### (4-3) 教師データに存在しない、単語１語だけの質問文

類似度が0.5を下回る結果となります。

In [17]:
'''
    こんにちは（正解＝なし）
'''
predict('こんにちは', _bot_id)

(['こんにちは'],
 [('4678_3855', 0.18185651302337646),
  ('4678_1848', 0.17963336408138275),
  ('4678_1847', 0.17834149301052094),
  ('4678_3856', 0.1740313172340393),
  ('4808_82', 0.1726617068052292),
  ('4808_175', 0.16706125438213348),
  ('4678_1226', 0.16010412573814392),
  ('4678_308', 0.1553257256746292),
  ('4808_88', 0.15404954552650452),
  ('4678_3234', 0.15360639989376068)])

### (4-4) 正解は教師データにないが、一部品詞が合致している質問文

4821、4837といったところが候補となり、類似度が0.5を上回る結果となります。

In [18]:
'''
    よいソフトがあれば見たい（正解＝なし）
'''
predict('よいソフトがあれば見たい', _bot_id)

(['よい', 'ソフト', '見る'],
 [('4837_18', 0.5997801423072815),
  ('4837_09', 0.5964978933334351),
  ('4837_02', 0.5954316258430481),
  ('4837_03', 0.5942075252532959),
  ('4837_10', 0.5915302038192749),
  ('4837_01', 0.5886266231536865),
  ('4837_11', 0.5882911682128906),
  ('4837_00', 0.5872790813446045),
  ('4837_19', 0.5868154168128967),
  ('4821_40', 0.5860151648521423)])

#### ご参考：教師データと一部品詞が合致する例

- '使いたいソフトがある'（正解＝4837）['使う', 'ソフト']

- '表計算ソフトについて'（正解＝4821）['表計算', 'ソフト']

### (4-5) 正解が教師データに存在せず、合致する品詞も無い質問文

類似度は、0.5を大幅に下回る結果となります。

In [19]:
'''
    おいしいラーメンが食べたいです（正解＝なし）
'''
predict('おいしいラーメンが食べたいです', _bot_id)

(['おいしい', 'ラーメン', '食べる'],
 [('4678_2906', 0.268244206905365),
  ('4678_898', 0.26373547315597534),
  ('4678_2907', 0.2627856135368347),
  ('4678_899', 0.2614479064941406),
  ('4709_04', 0.2589472234249115),
  ('4709_13', 0.2562527656555176),
  ('4707_04', 0.25339412689208984),
  ('4707_02', 0.2533625364303589),
  ('4678_3132', 0.2519910931587219),
  ('4678_1124', 0.24852317571640015)])